In [1]:
!nvidia-smi

Tue Sep 24 21:05:07 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1A:00.0 Off |                  N/A |
| 30%   24C    P8     7W / 250W |     11MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:1B:00.0 Off |                  N/A |
| 31%   26C    P8    22W / 250W |     11MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [2]:
from keras.utils.vis_utils import plot_model
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, LSTM, TimeDistributed, Dense, RepeatVector, CuDNNLSTM, GRU, Bidirectional, Input, CuDNNGRU
from keras.utils import np_utils
from keras.callbacks import TensorBoard
import tensorflow as tf
import os
from keras import backend as K
from keras.models import Model
from keras.layers.core import Dense, Reshape
from keras.layers.wrappers import TimeDistributed
from keras.layers import concatenate
import difflib
from keras.models import load_model
import keras
from keras import losses
import matplotlib.pyplot as plt
import random
from random import choice
import re

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle


# In[3]:


os.environ["CUDA_VISIBLE_DEVICES"]="0"
seq_length = 10
val_loss_hist = []

K.clear_session()
keras.backend.clear_session()

X_train=np.load('prepData/X_train_gap_hg38_v3_chr2_size10.npy')
X_val=np.load('prepData/X_val_gap_hg38_v3_chr2_size10.npy')
X_test=np.load('prepData/X_test_gap_hg38_v3_chr2_size10.npy')
y_train=np.load('prepData/y_train_gap_hg38_v3_chr2_size10.npy')
y_val=np.load('prepData/y_val_gap_hg38_v3_chr2_size10.npy')
y_test=np.load('prepData/y_test_gap_hg38_v3_chr2_size10.npy')

y_train1 = np.load('prepData/y_train1_gap_hg38_v3_chr2_size10.npy')
y_val1 = np.load('prepData/y_val1_gap_hg38_v3_chr2_size10.npy')
y_test1 = np.load('prepData/y_test1_gap_hg38_v3_chr2_size10.npy')

Using TensorFlow backend.


In [3]:
print(X_train)

[[[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 1 0]
  [0 0 0 1 0]
  [0 0 0 1 0]]

 [[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 1 0]
  [0 0 0 1 0]
  [0 0 0 0 1]]

 [[1 0 0 0 0]
  [1 0 0 0 0]
  [1 0 0 0 0]
  ...
  [0 0 0 1 0]
  [0 0 0 0 1]
  [0 0 0 0 1]]

 ...

 [[0 1 0 0 0]
  [1 0 0 0 0]
  [0 1 0 0 0]
  ...
  [1 0 0 0 0]
  [0 1 0 0 0]
  [1 0 0 0 0]]

 [[1 0 0 0 0]
  [0 1 0 0 0]
  [1 0 0 0 0]
  ...
  [0 1 0 0 0]
  [1 0 0 0 0]
  [0 1 0 0 0]]

 [[0 1 0 0 0]
  [1 0 0 0 0]
  [0 1 0 0 0]
  ...
  [1 0 0 0 0]
  [0 1 0 0 0]
  [1 0 0 0 0]]]


In [ ]:
nucleotide = ['0', 'A', 'C', 'G', 'T', '-']
def decoder(array):
    result = ""
    size = len(array)
    for i in range(size):
        if array[i].tolist() == [1, 0, 0, 0, 0, 0]:
            result=result+"0" 
        elif array[i].tolist() == [0, 1, 0, 0, 0, 0]:
            result=result+"A"
        elif array[i].tolist() == [0, 0, 1, 0, 0, 0]:
            result=result+"C"
        elif array[i].tolist() == [0, 0, 0, 1, 0, 0]:
            result=result+"G"
        elif array[i].tolist() == [0, 0, 0, 0, 1, 0]:
            result=result+"T"
        elif array[i].tolist() == [0, 0, 0, 0, 0, 1]:
            result=result+"-"
    return result

#model5 = load_model('model/seq2seq_nogap_camFam3_1mut.h5')
def decoderY(array):
    result = ""
    size = len(array)
    
    if array.tolist() == [1, 0, 0, 0, 0, 0]:
        result=result+"0" 
    elif array.tolist() == [0, 1, 0, 0, 0, 0]:
        result=result+"A"
    elif array.tolist() == [0, 0, 1, 0, 0, 0]:
        result=result+"C"
    elif array.tolist() == [0, 0, 0, 1, 0, 0]:
        result=result+"G"
    elif array.tolist() == [0, 0, 0, 0, 1, 0]:
        result=result+"T"
    elif array.tolist() == [0, 0, 0, 0, 0, 1]:
        result=result+"-"
    return result


def printHitMiss(a,b):
    if a==b:
        return 'Hit'
    else:
        return 'Miss'
    
print(X_test.shape)
def accuracy(a, b):
    count = 0
    for i in range(len(a)):
        if a[i] == b[i]:
            count = count+1
    return count/len(a)

def accuracy2(a, b, c):
    count = 0
    count2 =0
    for i in range(len(a)):
        if a[i] != c[i]:
            count2 = count2 +1
        if a[i] != c[i] and b[i]==c[i]:
            count = count+1
    return count/count2

def isMutation(a, b):
    if a!= b:
        print("mutation")

def predict(model):
    x_true=[]
    y_hat_list = []
    y_true = []
    predictions = model.predict(X_test, batch_size=250, verbose=0)
    
    for i, prediction in enumerate(predictions):
        #print(prediction)
        # print(prediction)
        #x_index = np.argmax(testX[i], axis=1)
        #print(prediction[i])
        x_str = decoder(X_test[i])

        #index = np.argmax(prediction)
        index = np.random.choice(6, 3, p=prediction)
        result = [nucleotide[index]]

        print(''.join(x_str), ' -> ', ''.join(result),
              " true: ", ''.join(decoderY(y_test[i])), printHitMiss(''.join(result), ''.join(decoderY(y_test[i]))))
        x_true.append(''.join(x_str[5]))
        y_hat_list.append(''.join(result))
        y_true.append(''.join(decoderY(y_test[i])))
    sm=difflib.SequenceMatcher(None,y_hat_list,y_true)
    sm2=difflib.SequenceMatcher(None,y_hat_list,x_true)
    print()
    print("Percentage of target and prediction being identical: {}".format(accuracy(y_hat_list, y_true)))
    print("Percentage of training and prediction being identical: {}".format(accuracy(y_hat_list, x_true)))
    print("Accuracy given mutation happened : {}".format(accuracy2(x_true, y_hat_list, y_true)))
    return x_true, y_hat_list, y_true


# In[ ]:


def lstm_model(latent_dim, half):
    encoder_inputs = Input(shape=(None, 5))
    
    encoder = Bidirectional(CuDNNLSTM(half, return_state=True))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
    state_h = concatenate([forward_h, backward_h])
    state_c = concatenate([forward_c, backward_c])
    
    
    # only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder
    decoder_inputs = Input(shape=(None, 5))
    decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_dense = Dense(5, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # inference
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

    # Run training
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']
                  )
    return model, encoder_model, decoder_model


def modelFit(epoch, batchSize, latent_dim, half, X_train, y_train, y_train1):
    model1, encoder_model1, decoder_model1 = lstm_model(latent_dim, half)
    hist1 = model1.fit([X_train, y_train1], y_train,
          batch_size=batchSize,
          epochs=epoch,
          validation_data=([X_val,y_val1], y_val),
          verbose = 2
         )
    return hist1, model1, encoder_model1, decoder_model1

def grid_search(latent, half,train_size, X_train, y_train, y_train1):
    hist1, model1, encoder_model1, decoder_model1 = modelFit(1, 100, latent, half, X_train, y_train, y_train1)
    hist2 ,model2, encoder_model2, decoder_model2 = modelFit(2, 100, latent, half, X_train, y_train, y_train1)
    hist3 ,model3, encoder_model3, decoder_model3 = modelFit(10, 100, latent, half, X_train, y_train, y_train1)
    #hist4 ,model4, encoder_model4, decoder_model4 = modelFit(30, 100, latent, half, X_train, y_train, y_train1)
    #hist5 ,model5, encoder_model5, decoder_model5 = modelFit(50, 100, latent, half, X_train, y_train, y_train1)
    #hist6 ,model6, encoder_model6, decoder_model6 = modelFit(80, 100, latent, half, X_train, y_train, y_train1)
    #hist7 ,model7, encoder_model7, decoder_model7 = modelFit(100, 100, latent, half, X_train, y_train, y_train1)
    #hist8 ,model8, encoder_model8, decoder_model8 = modelFit(500, 100, latent, half)

    model1.save("models/gap_hg38_v2_{}_{}_1.h5".format(train_size,half))
    model2.save("models/gap_hg38_v2_{}_{}_2.h5".format(train_size,half))
    model3.save("models/gap_hg38_v2_{}_{}_10.h5".format(train_size,half))
    #model4.save("models/_gap_hg38_v2_{}_{}_30_double.h5".format(train_size,half))
    #model5.save("models/_gap_hg38_v2_{}_{}_50_double.h5".format(train_size,half))
    #model6.save("models/_gap_hg38_v2_{}_{}_80_double.h5".format(train_size,half))
    #model7.save("models/_gap_hg38_v2_{}_{}_100_double.h5".format(train_size,half))
    #model8.save("_gap_hg38_v2_{}_{}_500.h5".format(train_size,half))
    
    encoder_model1.save("models/Egap_hg38_v2_{}_{}_1.h5".format(train_size,half))
    encoder_model2.save("models/Egap_hg38_v2_{}_{}_2.h5".format(train_size,half))
    encoder_model3.save("models/Egap_hg38_v2_{}_{}_10.h5".format(train_size,half))
    #encoder_model4.save("models/E_gap_hg38_v2_{}_{}_30_double.h5".format(train_size,half))
    #encoder_model5.save("models/E_gap_hg38_v2_{}_{}_50_double.h5".format(train_size,half))
    #encoder_model6.save("models/E_gap_hg38_v2_{}_{}_80_double.h5".format(train_size,half))
    #encoder_model7.save("models/E_gap_hg38_v2_{}_{}_100_double.h5".format(train_size,half))
    #encoder_model8.save("E_gap_hg38_v2_{}_{}_500.h5".format(train_size,half))
    
    decoder_model1.save("models/Dgap_hg38_v2_{}_{}_1.h5".format(train_size,half))
    decoder_model2.save("models/Dgap_hg38_v2_{}_{}_2.h5".format(train_size,half))
    decoder_model3.save("models/Dgap_hg38_v2_{}_{}_10.h5".format(train_size,half))
    #decoder_model4.save("models/D_gap_hg38_v2_{}_{}_30_double.h5".format(train_size,half))
    #decoder_model5.save("models/D_gap_hg38_v2_{}_{}_50_double.h5".format(train_size,half))
    #decoder_model6.save("models/D_gap_hg38_v2_{}_{}_80_double.h5".format(train_size,half))
    #decoder_model7.save("models/D_gap_hg38_v2_{}_{}_100_double.h5".format(train_size,half))
    #decoder_model8.save("D_gap_hg38_v2_{}_{}_500.h5".format(train_size,half))
    
    count = [i for i in range(len(hist3.history['val_loss']))]
    val_loss_hist.append([hist3.history['val_loss'].index(min(hist3.history['val_loss'])),min(hist3.history['val_loss'])])
    print(val_loss_hist)
    for i, value in zip(count, hist3.history['val_loss']):
        print(i, value)

grid_search(16, 8, 000, X_train, y_train, y_train1)        
grid_search(32, 16, 000, X_train, y_train, y_train1)
grid_search(64, 32, 000, X_train, y_train, y_train1)
grid_search(128, 64, 000, X_train, y_train, y_train1)
grid_search(256, 128, 000, X_train, y_train, y_train1)
grid_search(512, 256, 000, X_train, y_train, y_train1)
grid_search(1024, 512, 000, X_train, y_train, y_train1)
#grid_search(8192, 4096, 000, X_train, y_train, y_train1)

with open('loss_hist.txt', 'wb') as fp:
    pickle.dump(val_loss_hist, fp)

(50000, 10, 5)
Train on 2000000 samples, validate on 35000 samples
Epoch 1/1
 - 228s - loss: 0.4573 - acc: 0.8700 - val_loss: 0.3607 - val_acc: 0.9133
Train on 2000000 samples, validate on 35000 samples
Epoch 1/2
 - 225s - loss: 0.5177 - acc: 0.8454 - val_loss: 0.4107 - val_acc: 0.8971
Epoch 2/2
 - 225s - loss: 0.3737 - acc: 0.9030 - val_loss: 0.3558 - val_acc: 0.9126
Train on 2000000 samples, validate on 35000 samples
Epoch 1/10
 - 229s - loss: 0.4853 - acc: 0.8614 - val_loss: 0.3734 - val_acc: 0.9130
Epoch 2/10
 - 227s - loss: 0.3426 - acc: 0.9153 - val_loss: 0.3319 - val_acc: 0.9221
Epoch 3/10
 - 226s - loss: 0.3124 - acc: 0.9224 - val_loss: 0.3072 - val_acc: 0.9283
Epoch 4/10
 - 224s - loss: 0.2933 - acc: 0.9277 - val_loss: 0.2912 - val_acc: 0.9323
Epoch 5/10
 - 225s - loss: 0.2804 - acc: 0.9312 - val_loss: 0.2787 - val_acc: 0.9356
Epoch 6/10
 - 225s - loss: 0.2720 - acc: 0.9333 - val_loss: 0.2746 - val_acc: 0.9368
Epoch 7/10
 - 225s - loss: 0.2662 - acc: 0.9347 - val_loss: 0.2715 

/home/mcb/dlim63/miniconda3/envs/research/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer cu_dnnlstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'concatenate_1/concat:0' shape=(?, 16) dtype=float32>, <tf.Tensor 'concatenate_2/concat:0' shape=(?, 16) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/mcb/dlim63/miniconda3/envs/research/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer cu_dnnlstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'concatenate_3/concat:0' shape=(?, 16) dtype=float32>, <tf.Tensor 'concatenate_4/concat:0' shape=(?, 16) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
/home/mcb/dlim63/miniconda3/envs/research/lib/python3.6/site-package

[[9, 0.25946872557912554]]
0 0.37342648005911283
1 0.33193594907011303
2 0.3071597013941833
3 0.29119593236063207
4 0.27866011497165477
5 0.27458589879529816
6 0.2714666390844754
7 0.26547548719814845
8 0.26109861017337865
9 0.25946872557912554
Train on 2000000 samples, validate on 35000 samples
Epoch 1/1
 - 221s - loss: 0.3277 - acc: 0.9152 - val_loss: 0.2366 - val_acc: 0.9486
Train on 2000000 samples, validate on 35000 samples
Epoch 1/2
 - 221s - loss: 0.3619 - acc: 0.9044 - val_loss: 0.2676 - val_acc: 0.9398
Epoch 2/2
 - 220s - loss: 0.2432 - acc: 0.9418 - val_loss: 0.2317 - val_acc: 0.9493
Train on 2000000 samples, validate on 35000 samples
Epoch 1/10
 - 222s - loss: 0.3465 - acc: 0.9095 - val_loss: 0.2477 - val_acc: 0.9456
Epoch 2/10
 - 220s - loss: 0.2299 - acc: 0.9457 - val_loss: 0.2275 - val_acc: 0.9497
Epoch 3/10
